# Yelp API - Lab


## Introduction 

We've seen how the Yelp API works and how to create basic visualizations using Folium. It's time to put those skills to work in order to create a working map! Taking things a step further, you'll also independently explore how to perform pagination in order to retrieve a full results set from the Yelp API!

## Objectives

You will be able to: 

* Using pagination and multiple functions, gather large amounts of data from an API, parse the data and make sense of it with meaningful analysis
* Create maps using Folium

## Problem Introduction

You've now worked with some API calls, but we have yet to see how to retrieve a more complete dataset in a programmatic manner. Returning to the Yelp API, the [documentation](https://www.yelp.com/developers/documentation/v3/business_search) also provides us details regarding the API limits. These often include details about the number of requests a user is allowed to make within a specified time limit and the maximum number of results to be returned. In this case, we are told that any request has a maximum of 50 results per request and defaults to 20. Furthermore, any search will be limited to a total of 1000 results. To retrieve all 1000 of these results, we would have to page through the results piece by piece, retrieving 50 at a time. Processes such as these are often referred to as pagination.

In this lab, you will define a search and then paginate over the results to retrieve all of the results. You'll then parse these responses as a DataFrame (for further exploration) and create a map using Folium to visualize the results geographically.

## Part I - Make the Initial Request

Start by making an initial request to the Yelp API. Your search must include at least 2 parameters: **term** and **location**. For example, you might search for pizza restaurants in NYC. The term and location is up to you but make the request below.

In [1]:
import json

def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [2]:
keys = get_keys("/Users/wvsharber/.secret/yelp_api.json")

api_key = keys['api_key']

In [14]:
import requests
term = 'pizza'
location = 'Tulsa OK'
SEARCH_LIMIT = 1

url = 'https://api.yelp.com/v3/businesses/search'

headers = {
        'Authorization': 'Bearer {}'.format(api_key),
    }

url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT
            }
response = requests.get(url, headers=headers, params=url_params)

In [4]:
print(response)
print(type(response.text))
print(response.text[:1000])

<Response [200]>
<class 'str'>
{"businesses": [{"id": "FVzl8rDPiTWEtrNEuCu-Xg", "alias": "storyville-coffee-company-seattle-9", "name": "Storyville Coffee Company", "image_url": "https://s3-media3.fl.yelpcdn.com/bphoto/nJgiyjMZ7sglAtc5wyKSLQ/o.jpg", "is_closed": false, "url": "https://www.yelp.com/biz/storyville-coffee-company-seattle-9?adjust_creative=GfrfRbyX2iYO3M0rvKOQeg&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=GfrfRbyX2iYO3M0rvKOQeg", "review_count": 1585, "categories": [{"alias": "coffee", "title": "Coffee & Tea"}], "rating": 4.5, "coordinates": {"latitude": 47.6088847, "longitude": -122.3403957}, "transactions": ["pickup", "delivery"], "price": "$$", "location": {"address1": "94 Pike St", "address2": "Ste 34", "address3": "Pike Place Market", "city": "Seattle", "zip_code": "98101", "country": "US", "state": "WA", "display_address": ["94 Pike St", "Ste 34", "Pike Place Market", "Seattle, WA 98101"]}, "phone": "+12067805777", "display_phone": "(206) 78

In [15]:
response.json()['total']

351

## Pagination

Now that you have an initial response, you can examine the contents of the JSON container. For example, you might start with ```response.json().keys()```. Here, you'll see a key for `'total'`, which tells you the full number of matching results given your query parameters. Write a loop (or ideally a function) which then makes successive API calls using the offset parameter to retrieve all of the results (or 5000 for a particularly large result set) for the original query. As you do this, be mindful of how you store the data. Your final goal will be to reformat the data concerning the businesses themselves into a pandas DataFrame from the json objects.

**Note: be mindful of the API rate limits. You can only make 5000 requests per day and are also can make requests too fast. Start prototyping small before running a loop that could be faulty. You can also use time.sleep(n) to add delays. For more details see https://www.yelp.com/developers/documentation/v3/rate_limiting.**

In [7]:
# Your code here; use a function or loop to retrieve all the results from your original request

#write for loop that gets 50 results at a time until 3400 (may need to limit how quickly each get request is sent)
#include error processing
#process the business part of the response into a pandas DataFrame
#concatenate each 50 results onto the DataFrame



In [89]:
import pandas as pd
import time

def get_yelp_results(searchterm, searchlocation):
    """This function returns the search results in a Pandas DataFrame
       from Yelp api based on a search term and search location"""
    term = searchterm
    location = searchlocation
    SEARCH_LIMIT = 50
    offset = 0
    
    url = 'https://api.yelp.com/v3/businesses/search'

    headers = {
        'Authorization': 'Bearer {}'.format(api_key),
        }

    url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset': offset
                }
    
    initialresponse = requests.get(url, headers=headers, params=url_params)
    businesses = pd.DataFrame.from_dict(initialresponse.json()['businesses'])
    while len(businesses) < initialresponse.json()['total'] and len(businesses) < 1000 :
        offset += 50
        url_params = {
                'term': term.replace(' ', '+'),
                'location': location.replace(' ', '+'),
                'limit': SEARCH_LIMIT,
                'offset': offset
                }
        response = requests.get(url, headers=headers, params=url_params)
        if response.status_code == 200:
            bus50 = pd.DataFrame.from_dict(response.json()['businesses'])
            businesses = pd.concat([businesses, bus50], ignore_index = True)
            print(offset)
            time.sleep(1)

        else:
            print("Error: unable to retrieve businesses. Server responded with status code", response.status_code)
            break
    
    return businesses
        

## Exploratory Analysis

Take the restaurants from the previous question and do an initial exploratory analysis. At minimum, this should include looking at the distribution of features such as price, rating and number of reviews as well as the relations between these dimensions.

In [ ]:
# Your code here

## Mapping

Look at the initial Yelp example and try and make a map using Folium of the restaurants you retrieved. Be sure to also add popups to the markers giving some basic information such as name, rating and price.

In [ ]:
# Your code here

## Summary

Nice work! In this lab, you've made multiple API calls to Yelp in order to paginate through a results set, performing some basic exploratory analysis and then creating a nice interactive map to display the results using Folium! Well done!